In [ ]:
import pandas as pd
import os
import gzip
import pickle
import openai
import re
import copy

In [ ]:
with open('hardware.txt', 'r') as file:
    hardware_txt = file.read()

# [x.split(';') for x in hardware_txt.split('\n')]
stuff = []
for x in hardware_txt.split('\n'):
    if len(x.split(';')) == 4:
        print("error")
        print(x)
    stuff.append(x.split(';'))

hardware_df = pd.DataFrame(stuff, columns=['hardware_name', 'hashrate', 'efficiency'])


#remove rows that contain x2,x3 etc
hardware_df = hardware_df[~hardware_df['hardware_name'].str.contains("x[0-9]")]
hardware_df = hardware_df[~hardware_df['hardware_name'].str.contains("cards")]

#remove text in brackets from hardware_name
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"\(.*\)","", x).strip())
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"OC","", x).strip())
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"\d+ *Gh/s","", x).strip())
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"\d+ *GH/S","", x).strip())
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"\d+ *GH/s","", x).strip())

#remove duplicate hardware names
hardware_df = hardware_df.drop_duplicates(subset=['hardware_name'])

#reset index
hardware_df = hardware_df.reset_index(drop=True)
hardware_df["hardware_index"] = hardware_df.index
hardware_df.to_csv('hardware_bitcoinwiki.csv', index=False)

# save it as a csv with columns "index,hardware_name"
hardware_df = hardware_df[['hardware_index', 'hardware_name']]
# hardware_df.to_csv('hardware_index.csv', index=False)

In [ ]:
with open('hardware_new.txt', 'r') as file:
    hardware_txt = file.read()

hardware_df = pd.DataFrame([x.split(';') for x in hardware_txt.split('\n')], columns=['hardware_name', 'date', 'speed','power','noise','hash','profit'])

# keep only SHA-256
hardware_df = hardware_df[hardware_df['hash'] == "SHA-256"]

# efficiency = speed/power
hardware_df['Mhash/J'] = hardware_df['speed'].str.replace("Th/s","").astype(float)/hardware_df['power'].str.replace("W","").astype(float) * 1000000

#remove text in brackets from hardware_name
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"\(.*\)","", x).strip())
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"OC","", x).strip())
hardware_df['hardware_name'] = hardware_df['hardware_name'].apply(lambda x: re.sub(r"\d+ *Th/s","", x).strip())

# rename date to hardware_release_date
hardware_df = hardware_df.rename(columns={"date": "hardware_release_date"})

#reset index
hardware_df = hardware_df.reset_index(drop=True)
hardware_df["hardware_index"] = hardware_df.index
hardware_df.to_csv('hardware_asicminervalue.csv', index=False)

with_date = hardware_df[['hardware_index', 'hardware_name', 'hardware_release_date']]
with_date.to_csv('asicminervalue_with_date.csv', index=False)

# save it as a csv with columns "index,hardware_name"
hardware_df = hardware_df[['hardware_index', 'hardware_name']]